In [1]:
library(ggstatsplot)
library(ggside)
library(dplyr)
library(summarytools)
library(ggfortify)
library(plyr)
library(purrr)

You can cite this package as:
     Patil, I. (2021). Visualizations with statistical details: The 'ggstatsplot' approach.
     Journal of Open Source Software, 6(61), 3167, doi:10.21105/joss.03167

Loading required package: ggplot2

Registered S3 method overwritten by 'ggside':
  method from   
  +.gg   ggplot2


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system2("/usr/bin/otool", c("-L", shQuote(DSO)), stdout = TRUE):
“running command ''/usr/bin/otool' -L '/Users/suzheng/opt/anaconda3/envs/r4new/lib/R/library/tcltk/libs//tcltk.so'' had status 1”
system might not have X11 capabilities; in case of errors when using dfSummary(), set st_options(use.x11 = FALSE)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you 

In [2]:
dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/cor_with_clinvar/anal/"
setwd(dir)

In [3]:
load("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/data/psi_meta.Rdata")
psi_dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/"


In [4]:
len_psi <- read.csv("../all.target_gene_psi.pasted.added_gene_symbols.clean", header=T, sep="\t", check.names=F)

In [5]:
gene_info <- len_psi[,1:4]
gene_info <- gene_info[match(rownames(psi_reordered), gene_info$exon),]

In [7]:
exon_coords <- read.csv("../gencode.v38.annotation.Exonic_Parts.LRG.noDup.gtf.exon_coord.target_genes", header=F, sep="\t")

In [8]:
colnames(exon_coords) <- c("chr", "start", "end", "exon")

In [9]:
gene_info.coord <- left_join(gene_info, exon_coords, by=c("exon0"="exon"))

In [10]:
gene_info.coord$index <- 1:dim(gene_info.coord)[1]

In [11]:
clinvar.rt <- read.csv("../clinvar.vcf.selected_genes.tsv", header=T, sep="\t")

In [12]:
clinvar.rt %>% dim()
psi %>%  dim()

[1] 583056     13

[1] 19637  1420

In [13]:
clinvar_patho_types <- c("Pathogenic", "Likely_pathogenic", "Pathogenic/Likely_pathogenic", "Pathogenic|_other", "Pathogenic|_drug_response", "Likely_pathogenic|_drug_response", "Likely_pathogenic|_association", "Likely_pathogenic,_low_penetrance", "Likely_pathogenic|_risk_factor", "Pathogenic|_risk_factor", "Pathogenic/Likely_pathogenic|_other", "Pathogenic/Likely_pathogenic|_risk_factor", "Pathogenic|_association|_protective")

In [14]:
#gene_info.path_var_stat <- gene_info.coord[, c("chr", "start", "end")] %>% pmap_dfr(stat_path_var_in_one_exon)
#save(gene_info.path_var_stat, file = "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/data/gene_info.path_var_stat.Rdata")

In [15]:
gnomad.var <- read.csv("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/gnomAD_var_anal/all_genes.gnomAD_var.csv")

In [16]:
gnomad.var <- rename(gnomad.var, replace=c("Chromosome"="CHROM", "Position"="POS", "VEP.Annotation"="clin_sig"))

In [ ]:
stat_some_types_of_var_in_every_exon <- function(var_df, target_var_types, res_col_name){
    stat_var_in_one_exon <- function(chr, start, end){
        vars.df <- var_df %>% filter(CHROM==chr & POS>=start & POS<=end)
        path.num <- sum(vars.df$clin_sig %in% target_var_types)
        df <- data.frame(path.num=path.num)
        colnames(df)[1] <- res_col_name
        df
    }

    gene_info.var_stat <- gene_info.coord[, c("chr", "start", "end")] %>% pmap_dfr(stat_var_in_one_exon)
    gene_info.var_stat
}

In [18]:
clinvar.path_var <-  stat_some_types_of_var_in_every_exon(clinvar.rt, clinvar_patho_types, "clinvar.path_var")
gnomad.missense <-  stat_some_types_of_var_in_every_exon(gnomad.var, c("missense_variant"), "gnomad.missense")
gnomad.syn <-  stat_some_types_of_var_in_every_exon(gnomad.var, c("synonymous_variant"), "gnomad.syn")
gnomad.frameshift <-  stat_some_types_of_var_in_every_exon(gnomad.var, c("frameshift_variant"), "gnomad.frameshift")
gnomad.missense_inframe <-  stat_some_types_of_var_in_every_exon(gnomad.var, c("missense_variant", "inframe_deletion", "inframe_insertion"), "gnomad.missense_inframe")
gnomad.frameshift_stopgain <-  stat_some_types_of_var_in_every_exon(gnomad.var, c("frameshift_variant", "stop_gained"), "gnomad.frameshift_stopgain")

gene_info.stat <- cbind(gene_info.coord, 
                        clinvar.path_var, 
                        gnomad.missense, 
                        gnomad.syn, 
                        gnomad.frameshift, 
                        gnomad.missense_inframe,
                        gnomad.frameshift_stopgain
                       )


In [ ]:

gene_info.stat.file = "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/data/gene_info.exon_var_stat.Rdata"
save(gene_info.stat, file = gene_info.stat.file)



In [20]:
write.table(gene_info.stat, file = paste0(gene_info.stat.file, ".tsv"), sep="\t", quote=F, col.names=NA)